In [1]:
import os
import math
import random
from typing import Dict, Literal, List, Set, Tuple, Iterable

import numpy as np
import pandas as pd
import networkx as nx
from scipy.stats import ttest_ind
from statsmodels.stats.multitest import fdrcorrection
import matplotlib.pyplot as plt

files = os.listdir('DataSujetos/')

In [11]:
ESTADIOS = ['W', 'N1', 'N2', 'N3']
NUM_SUJETOS = 18
ADJ_MATRIX_SIZE = 116
GLOBAL_SEED = 158151
PC = 0.05
ZC = 1

In [2]:
class Experiment():
    
    def __init__(self, density: float):
        self.density = density
        self._data = {
            estadio: {
                f'suj{i+1}': {
                    'zi': np.zeros(ADJ_MATRIX_SIZE),
                    'Pi': np.zeros(ADJ_MATRIX_SIZE),
                } for i in range(NUM_SUJETOS)
            } for estadio in ESTADIOS
         }
        
    def set_experiment_data(self, 
                            data: Dict[str, Iterable[float]], 
                            estadio: Literal['W', 'N1', 'N2', 'N3'], 
                            sujeto: str) -> None:
        self._data[estadio][sujeto] = data
            
    def get_node_classification_data_by_estadio(self, 
                                                estadio: Literal['W', 'N1', 'N2', 'N3']
                                               ) -> Dict[str, Dict[str, int]]:
        relevant_data = self._data[estadio]
        ans = {}
        for sujeto, raw_data in relevant_data.items():
            zi_mask = raw_data['zi'] > ZC
            Pi_mask = raw_data['Pi'] > PC
            num_Provincial_Hubs = np.sum(zi_mask & ~Pi_mask)
            num_Hubs = np.sum(zi_mask & Pi_mask)
            num_Connector_Nodes = np.sum(~zi_mask & Pi_mask)
            num_Provincial_Nodes = np.sum(~zi_mask & ~Pi_mask)
            ans[sujeto] = {
                'Provincial_Hubs': num_Provincial_Hubs,
                'Hubs': num_Hubs,
                'Connector_Nodes': num_Connector_Nodes,
                'Provincial_Nodes': num_Provincial_Nodes,
            }
        return ans

In [3]:
def find_threshold(adj_matrix: np.ndarray, density: float, epsilon: int=2) -> float:
    size = adj_matrix.shape
    N = size[0]*size[1]
    left = adj_matrix.min()
    right = adj_matrix.max()
    target = math.ceil(N*density)
    res = N
    cnt = 0
    limit = 100
    while res!=target:
        cnt+=1
        if cnt == limit:
            print(f'Have not converged in {limit} iterations, returning suboptimal threshold')
            return guess
        guess = (left+right)/2
        res = np.sum(adj_matrix>=guess)
        if (res>=target-epsilon) and (res<=target+epsilon):
            return guess
        elif res<target:
            right = guess
        else:
            left = guess

            
def extract_sujeto(filename: str) -> str:
    return filename.split('.')[0].split('_')[1]


def _compute_zi(G: nx.Graph) -> Dict[int, float]:
    degrees = G.degree()
    k_mean = np.mean([degree for node, degree in degrees])
    k_std = np.std([degree for node, degree in degrees])
    return {node: (degree-k_mean)/k_std for node, degree in degrees}


def _compute_Pi(G: nx.Graph, comms: List[Set[int]]) -> Dict[int, float]:
    # We start with the initial value of 1 and iteratively subtract from it according to the definition
    Pi = {node: 1 for node in range(ADJ_MATRIX_SIZE)}
    degrees = G.degree()
    for node in Pi:
        ki = degrees[node]
        neighbors = set(nx.all_neighbors(G, node))
        for C in comms:
            kiUj = len(neighbors.intersection(C))
            tmp = (kiUj/ki)**2 if ki>0 else 0
            Pi[node] -= tmp
    return Pi


def run_node_experiment(G: nx.Graph) -> Dict[str, Dict[int, float]]:
    comms = nx.community.louvain_communities(G, seed=GLOBAL_SEED)
    zi = _compute_zi(G)
    zi_values = np.array([x for x in zi.values()])
    Pi = _compute_Pi(G, comms)
    Pi_values = np.array([x for x in Pi.values()])
    return {'zi': zi_values, 'Pi': Pi_values}
    
    
def graph_factory(adjmat: np.ndarray) -> nx.Graph:
    adjmat -= np.diag(np.diag(adjmat))
    thr = find_threshold(adjmat, d)
    adjmat = adjmat>=thr
    G = nx.from_numpy_array(adjmat)
    return G

In [4]:
%%time

densities = np.linspace(0.01, 0.15, num=15)
experiments = [Experiment(d) for d in densities]
avg_graphs = []
running_avg_graph = {
    estadio: np.zeros((ADJ_MATRIX_SIZE, ADJ_MATRIX_SIZE)) for estadio in ESTADIOS
}

for i, d in enumerate(densities):
    print(f'Start with density {d}')
    exp = experiments[i]
    for estadio in ESTADIOS:
        for f in files:
            if not f.startswith(estadio):
                continue
            sujeto = extract_sujeto(f)
            adjmat = pd.read_csv('DataSujetos/'+f, header=None).values
            running_avg_graph[estadio] += adjmat/NUM_SUJETOS
            G = graph_factory(adjmat)
            node_data = run_node_experiment(G)
            exp.set_experiment_data(node_data, estadio, sujeto)
    avg_graphs.append(running_avg_graph)
    running_avg_graph = {
        estadio: np.zeros((ADJ_MATRIX_SIZE, ADJ_MATRIX_SIZE)) for estadio in ESTADIOS
    }

Start with density 0.01
Start with density 0.019999999999999997
Start with density 0.03
Start with density 0.039999999999999994
Start with density 0.049999999999999996
Have not converged in 100 iterations, returning suboptimal threshold
Start with density 0.05999999999999999
Start with density 0.06999999999999999
Start with density 0.07999999999999999
Start with density 0.08999999999999998
Start with density 0.09999999999999998
Start with density 0.10999999999999997
Start with density 0.11999999999999998
Start with density 0.12999999999999998
Start with density 0.13999999999999999
Start with density 0.15
CPU times: total: 20.3 s
Wall time: 20.3 s


In [5]:
exp = experiments[5]

In [6]:
exp.get_node_classification_data_by_estadio('N1')

{'suj1': {'Provincial_Hubs': 5,
  'Hubs': 16,
  'Connector_Nodes': 37,
  'Provincial_Nodes': 58},
 'suj2': {'Provincial_Hubs': 3,
  'Hubs': 18,
  'Connector_Nodes': 47,
  'Provincial_Nodes': 48},
 'suj3': {'Provincial_Hubs': 7,
  'Hubs': 20,
  'Connector_Nodes': 30,
  'Provincial_Nodes': 59},
 'suj4': {'Provincial_Hubs': 0,
  'Hubs': 17,
  'Connector_Nodes': 57,
  'Provincial_Nodes': 42},
 'suj5': {'Provincial_Hubs': 1,
  'Hubs': 13,
  'Connector_Nodes': 52,
  'Provincial_Nodes': 50},
 'suj6': {'Provincial_Hubs': 5,
  'Hubs': 21,
  'Connector_Nodes': 35,
  'Provincial_Nodes': 55},
 'suj7': {'Provincial_Hubs': 0,
  'Hubs': 23,
  'Connector_Nodes': 50,
  'Provincial_Nodes': 43},
 'suj8': {'Provincial_Hubs': 2,
  'Hubs': 21,
  'Connector_Nodes': 34,
  'Provincial_Nodes': 59},
 'suj9': {'Provincial_Hubs': 2,
  'Hubs': 21,
  'Connector_Nodes': 38,
  'Provincial_Nodes': 55},
 'suj10': {'Provincial_Hubs': 5,
  'Hubs': 15,
  'Connector_Nodes': 33,
  'Provincial_Nodes': 63},
 'suj11': {'Provinc

In [10]:
avg_graphs[0]['W'].shape

(116, 116)